In [ ]:
from os import chdir
chdir("../")
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import MinMaxScaler
from matplotlib import pyplot as plt
import seaborn as sns
from src.model.dp_ppms_model import estim_bayes_theta, clustering_2

In [ ]:
data = pd.read_csv("./data/mcs_ds_edited_iter_shuffled.csv", sep=",")
y = data.ale
x = data[["anchor_ratio", "trans_range", "node_density", "iterations"]]

In [ ]:
data.head(5)

Dans le but de calculer la loss $SC_{ k1,k2,k3}$, il faut calculer les estimateurs $\hat{\eta}_{B}$ et $\hat{\eta}_{\phi}$. Les fonctions suivantes sont faites dans le but de calculer respectivement $\hat{\theta}_{B}$, $\hat{\beta}_{B}$, $\hat{\sigma}^{2}_{B}$, $\hat{\theta}_{\phi}$, $\hat{\beta}_{\phi}$, $\hat{\sigma}^{2}_{\phi}$ de l'algorithme. Pour le faire, nous calculons les distribution a posteriori de $\eta | y$ et $\eta |y,\phi$ et nous déduisons l'espérance de ces distributions. Les fonctions ci-dessous sont faites après calcul à la main de ces distributions. Après calcul, nous obtenons: 

- $\beta|y \sim \mathcal{N} \left(  \left(n\bar{y}-\theta_{k}\right)1_{n}-\beta_{0},\sigma_{0}^{2}B_{0}x_{i}^{\top}x_{i} \right)$ où $1_{n}$ représente le vecteur constitué des 1, de même taille que $\beta_{0}$
- $\sigma^{2}|y \sim I\Gamma \left( \nu_{0}-\frac{n}{2},\sum_{i=1}^{n}(y_{i}-\theta_{k}-x_{i}^{\top}\beta)^{2}+\lambda_{0}\right)$

In [ ]:
theta = estim_bayes_theta(x,y,theta0=0,tau0=1000,phi = np.array([range(len(y))]),c=12)

In [ ]:
fig, axe =  plt.subplots(1, 1, figsize=(9, 5.5))
axe.plot(list(data.index), theta,linestyle='',marker='o')
axe.set_ylabel("value of $\\theta_j$")
axe.set_xlabel("index of sonser")
plt.savefig("theta.pgf", format='pgf')
plt.show()

In [ ]:
clustering_2(y,x,theta0=0,beta0=0,tau0=1000,lamda_0=2,nu_0=1,c=12,k1=0.001,k2=0.098,k3=0.9,epsilon=10^-1)